# Lecture 10
## Object Oriented Programming
### Wednesday, October 4th 2017

In [1]:
from IPython.display import HTML

## Recap
* We introduced the idea of objects in Python
* We discussed Python classes
* We went over **inheritance**
* We *briefly* touched on **polymorphism**
* We started to introduce Python special methods (the *dunder* methods)

## Example:  Printing with `__repr__` and `__str__`

* The way printing works is that Python wants classes to implement `__repr__` and `__str__` methods. 
* It will use inheritance to give the built-in `object`s methods when these are not defined.
* Any class can define `__repr__` and `__str__`. 
* When an *instance* of such a class is interrogated with the `repr` or `str` function, then these underlying methods are called.

We'll see `__repr__` here. If you define `__repr__` you have made an object sensibly printable.

####   `__repr__`  

In [27]:
class Animal():
    
    def __init__(self, name):
        self.name=name
        
    def __repr__(self):
        class_name = type(self).__name__
        return "{0!s}({1.name!r})".format(class_name, self)

In [28]:
r = Animal("David")
r

Animal('David')

In [29]:
print(r)

Animal('David')


In [30]:
repr(r)

"Animal('David')"

### The pattern with *dunder* methods


**There are functions without double-underscores that cause the methods with the double-underscores to be called**

Thus `repr(an_object)` will cause `an_object.__repr__()` to be called. 

In user-level code, you *SHOULD NEVER* see the latter. In library level code, you might see the latter. The definition of the class is considered library level code.

## Example:  Instance Equality via `__eq__`

Now we are in a position to answer the question:  What makes two objects equal?

To do this, we will add a new dunder method to the mix, the unimaginatively (thats a good thing) named `__eq__`.

In [1]:
class Animal():
    
    def __init__(self, name):
        self.name=name
        
    def __repr__(self):
        class_name = type(self).__name__
        return "{0!s}({1.name!r})".format(class_name, self)
    
    def __eq__(self, other):
        return self.name==other.name # two animals are equal if their names are equal

In [32]:
A=Animal("Tom")
B=Animal("Jane")
C=Animal("Tom")

There are three separate object identities, but we made two of them equal!

In [33]:
print(id(A), "   ", id(B), "   ", id(C))

print(A==B, "         ", B==C, "         ", A==C)

4550671216     4550668528     4550668808
False           False           True


This is critical because it gives us a say in what equality means.

## Python's power comes from the data model, composition, and delegation

The data model is used (from **Fluent Python**) to provide a:

>description of the interfaces of the building blocks of the language itself, such as sequences, iterators, functions, classes....

The special "dunder" methods we talk about are invoked by the `Python` interpreter to beform basic operations. 

For example, `__getitem__` gets an item in a sequence. This is used to do something like `a[3]`. 

`__len__` is used to say how long a sequence is. Its invoked by the `len` built-in function. 

A **sequence**, for example,  must implement `__len__` and `__getitem__`. Thats it.

The original reference for this data model is: https://docs.python.org/3/reference/datamodel.html .

### Tuple

An example of a sequence in Python is the tuple. Since a tuple is a sequence, it must support indexing and be able to tell us its length.

In [34]:
a=(1,2)
a[0]

1

In [35]:
len(a)

2

### NamedTuples

#### `collections.namedtuple`
* Produces subclasses of tuples
* The tuples are enhanced with field names and a class name.

Consider, the example from **Fluent Python** (Example 1-1):

In [7]:
import collections
Card = collections.namedtuple('Card', ['rank', 'suit'])
repr(Card)

"<class '__main__.Card'>"

In [4]:
my_card = Card('3', 'diamonds')
print(my_card)
print(type(my_card))
print(my_card.rank)

Card(rank='3', suit='diamonds')
<class '__main__.Card'>
3


#### A Custom Sequence

We now wish to create a `FrenchDeck` as an example of something that follows Python's Sequence protocol. Remember, the sequence protocol requires implementation of two methods: `__len__` and `__getitem__`. Thats it.

In [8]:
class FrenchDeck:
    ranks = [str(n) for n in range(2,11)] + list('JKQA')
    suits="spade diamond club heart".split()
    
    def __init__(self):
        # composition: there are items IN this class that constutute its structure
        # delegation: the storage for this class is DELEGATED to this list below
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

In [11]:
class FrenchDeck:
    ranks = [str(n) for n in range(2,11)] + list('JKQA')
    suits="spade diamond club heart".split()
    
    def __init__(self):
        # composition: there are items IN this class that constutute its structure
        # delegation: the storage for this class is DELEGATED to this list below
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

In [12]:
deck = FrenchDeck()
len(deck)

52

In [13]:
deck[0], deck[-1], deck[3]

(Card(rank='2', suit='spade'),
 Card(rank='A', suit='heart'),
 Card(rank='5', suit='spade'))

In [14]:
deck[10:15]

[Card(rank='K', suit='spade'),
 Card(rank='Q', suit='spade'),
 Card(rank='A', suit='spade'),
 Card(rank='2', suit='diamond'),
 Card(rank='3', suit='diamond')]

* The `FrenchDeck` class supports the sequence protocol
* As a result, we can use functions like `random.choice` *directly* on instances of `FrenchDeck`. 
* This is the power of interfaces and the data model.

In [15]:
from random import choice
choice(deck)

Card(rank='2', suit='club')

# What is Polymorphism?

We saw polymorphism last time as the ability to tun the same methods on different objects, either through inheritance or by just defining an ad-hoc protocol (duck typing).

The more general definition is:

**The ability to write code that looks similar, but operates on different types.**

In other words, a single interface serves entities of different types. 


## Polymorphism Summary

Python type system is strong and dynamic:

- strong: everything has a well-defined type: `type`, `isinstance`
- dynamic: type is not explicitly declared, changes with content

In classic dynamically typed languages (e.g. Python) most common code is polymorphic. 

The types of values are restricted only by explicit runtime checks or errors due to failed support for operations at run time. 

Polymorphism is often combined with inheritance, but does not need to be.

One classification of Polymorphism (summarized from Wikipedia) divides it as:
- on one axis:  adhoc, parametric, and subtype based
- on another axis:  dynamic (run time) and static (compile time)

### Static vs Dynamic

Dynamic (run-time) polymorphism can be thought of as table based dispatch: that there is, somewhere, atleast conceptually, a table of types, or a linkage of such tables created by inheritance, where implementations are looked up for types.

In static polymorphism (e.g. in `C++`) the binding to the appropriate class can be done at compile time.

### Back to our French Deck Example

In [16]:
import collections
Card = collections.namedtuple('Card', ['rank', 'suit'])

In [17]:
class FrenchDeck():
    ranks = [str(n) for n in range(2, 11)] + list('JQKA') 
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
                       
    def __len__(self):
        return len(self._cards)
                       
    def __getitem__(self, position): 
        return self._cards[position]
        

In [26]:
mydeck = FrenchDeck()
print(len(mydeck))

52


In [27]:
vars(mydeck)

{'_cards': [Card(rank='2', suit='spades'),
  Card(rank='3', suit='spades'),
  Card(rank='4', suit='spades'),
  Card(rank='5', suit='spades'),
  Card(rank='6', suit='spades'),
  Card(rank='7', suit='spades'),
  Card(rank='8', suit='spades'),
  Card(rank='9', suit='spades'),
  Card(rank='10', suit='spades'),
  Card(rank='J', suit='spades'),
  Card(rank='Q', suit='spades'),
  Card(rank='K', suit='spades'),
  Card(rank='A', suit='spades'),
  Card(rank='2', suit='diamonds'),
  Card(rank='3', suit='diamonds'),
  Card(rank='4', suit='diamonds'),
  Card(rank='5', suit='diamonds'),
  Card(rank='6', suit='diamonds'),
  Card(rank='7', suit='diamonds'),
  Card(rank='8', suit='diamonds'),
  Card(rank='9', suit='diamonds'),
  Card(rank='10', suit='diamonds'),
  Card(rank='J', suit='diamonds'),
  Card(rank='Q', suit='diamonds'),
  Card(rank='K', suit='diamonds'),
  Card(rank='A', suit='diamonds'),
  Card(rank='2', suit='clubs'),
  Card(rank='3', suit='clubs'),
  Card(rank='4', suit='clubs'),
  Card(r

In [28]:
dir(mydeck)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cards',
 'ranks',
 'suits']

In [30]:
vars(mydeck.__class__)

mappingproxy({'__dict__': <attribute '__dict__' of 'FrenchDeck' objects>,
              '__doc__': None,
              '__getitem__': <function __main__.FrenchDeck.__getitem__>,
              '__init__': <function __main__.FrenchDeck.__init__>,
              '__len__': <function __main__.FrenchDeck.__len__>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'FrenchDeck' objects>,
              'ranks': ['2',
               '3',
               '4',
               '5',
               '6',
               '7',
               '8',
               '9',
               '10',
               'J',
               'Q',
               'K',
               'A'],
              'suits': ['spades', 'diamonds', 'clubs', 'hearts']})

In [31]:
dir(my_deck.__class__)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'ranks',
 'suits']

## Ad hoc Polymorphism and Object tables

**Ad hoc polymorphism** is the notion that different functions are called to accomplish the same task for arguments of different types. 

This enables the Python Data model with the dunder methods. 

If you call `len(arg)` or `iter(arg)`, we delegate to `arg`'s `__len__` or `__iter__` by *looking them up in the table (class) corresponding to arg*. 

**The net effect is that you get different behaviors for different objects.**

You are not looking up a table for the operation but instead looking up a table for the object.

You can think of this as *single dispatch*: the `len` is dispatched based on the type of the argument by looking up a table for the argument.

### Duck Typing

- We group together the notion that an object responds to such "messages" into a protocol
- An example is the informal notion that something is a sequence

This is **Duck Typing**. 

Alex Martelli, the coiner of the phrase *Duck Typing*, says:

>In Python, this mostly boils down to avoiding the use of isinstance to check the object’s type (not to mention the even worse approach of checking, for example, whether type(foo) is bar—which is rightly anathema as it inhibits even the simplest forms of inheritance!).

### Tables for dispatching on functions

You can also dispatch a function based on its argument, with no lookup in that argument's table, but rather in a *table that is associated with the function*. This is also *single dispatch*, but from a different table.

There is no built in support in Python for this, but you can write it on your own by associating a dictionary with multiple types.

See Chapter 7 (Example 7-20 and Example 7-21) in *Fluent Python*.

## Parametric Polymorphism

**write functions (or types) that are generic "over" other types.**

- This means, for example, a stack that can take either an int or a float or an animal. Notice that this is generally true in a dynamic language such as Python where objects are allocated on the heap and its the references or labels or ids that are pushed onto the stack. 

- In C++ this can be done using templates at compile time to optimize the allocation of space, for example.

## Subtype Polymorphism

This refers to the polymorphism that we encounter in situations where our language provides subclassing.

- In a language such as C++, this refers to the notion that a dog and a cat can make sounds through an animal pointer. 
- In Python one can use duck typing or inheritance. So subtype polymorphism is then just ad-hoc polymorphism plus an augmented lookup in the inheritance hierarchy.

### Object Tables Again

What's this table we keep talking about? We hinted at it earlier when we did:

In [41]:
mydeck.__class__.__dict__

mappingproxy({'__dict__': <attribute '__dict__' of 'FrenchDeck' objects>,
              '__doc__': None,
              '__getitem__': <function __main__.FrenchDeck.__getitem__>,
              '__init__': <function __main__.FrenchDeck.__init__>,
              '__len__': <function __main__.FrenchDeck.__len__>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'FrenchDeck' objects>,
              'ranks': ['2',
               '3',
               '4',
               '5',
               '6',
               '7',
               '8',
               '9',
               '10',
               'J',
               'Q',
               'K',
               'A'],
              'suits': ['spades', 'diamonds', 'clubs', 'hearts']})

What if we dont find a method in the table?

Either this is a runtime error, or we search in the "parent" classes of this class.

We can see all such attributes by using `dir`:

In [43]:
dir(mydeck)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cards',
 'ranks',
 'suits']

This works because it gets sent up:

In [56]:
hash(mydeck)

273906717

You can see whats upward of the French Deck by inspecting the [*Method Order Resolution*](https://www.python.org/download/releases/2.3/mro/) using the `mro` method.

In [57]:
FrenchDeck.mro()

[__main__.FrenchDeck, object]

## ABCs

- Python actually has a more structured form of typing, called **Goosetyping**.
- Goosetyping is a generalization of duck typing with a bit more structure. 
- Goosetyping is used to define **protocols** such as the notion of a "sequence". 

Clearly, this can be done informally by agreement as well, but there is an advantage to defining things higher in an inheritance hierarchy and more formally.

Indeed we will soon use ***ABC***s as a vehicle to document interfaces. 

### Inheritance and ABCs
Back to our cards example from Fluent Python. 

In [35]:
class FrenchDeck():
    ranks = [str(n) for n in range(2, 11)] + list('JQKA') 
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
                       
    def __len__(self):
        return len(self._cards)
                       
    def __getitem__(self, position): 
        return self._cards[position]

In [36]:
issubclass(FrenchDeck, collections.abc.Sequence)

False

No we are not a sequence and thus:

In [38]:
deck=FrenchDeck()
deck.index(Card(rank='7', suit='spades'))

AttributeError: 'FrenchDeck' object has no attribute 'index'

* That's disappointing.  We defined `__getitem__` and `__len__` which are the necessary special methods for a sequence.

* We can go see the source code for the Sequence ABC at https://github.com/python/cpython/blob/master/Lib/_collections_abc.py#L862.

Some salient points:

- Notice that `__getitem__` is declared as abstract. This means that we must provide an implementation. Which we have! We did the same for `__len__`.  But implementing a protocol does not mean being a subclass.

- The source code seems to indicate that only `__getitem__` and `__len__` must be defined. But notice that we have `class Sequence(Sized, Iterable, Container)`. This is multiple Inheritance, where the other ABCs are being used to **Mixin** protocols that we want `Sequence` to support. So lets go digging:

In [39]:
abms=set()
abms = abms.union(collections.abc.Sequence.__abstractmethods__)
print(collections.abc.Sequence.mro())
print(collections.abc.Sequence.__abstractmethods__)
print('=======')
for parent in collections.abc.Sequence.mro()[1:-1]:
    print(parent.__name__, parent.mro())
    print(parent.__abstractmethods__)
    print('-----------')
    abms=abms.union(parent.__abstractmethods__)
abms

[<class 'collections.abc.Sequence'>, <class 'collections.abc.Sized'>, <class 'collections.abc.Iterable'>, <class 'collections.abc.Container'>, <class 'object'>]
frozenset({'__getitem__', '__len__'})
Sized [<class 'collections.abc.Sized'>, <class 'object'>]
frozenset({'__len__'})
-----------
Iterable [<class 'collections.abc.Iterable'>, <class 'object'>]
frozenset({'__iter__'})
-----------
Container [<class 'collections.abc.Container'>, <class 'object'>]
frozenset({'__contains__'})
-----------


{'__contains__', '__getitem__', '__iter__', '__len__'}

Notice that we get `__contains__` and `__iter__` for free. `__contains__` uses `__iter__` and `__iter__` needs `__getitem__`. But Our French Deck is not a sequence subclass.

Why are we fixating on that? Patience, little grasshopper...

#### Registration

We can explicitly register to make sure all is good:

In [40]:
collections.abc.Sequence.register(FrenchDeck)
deck = FrenchDeck()
issubclass(FrenchDeck, collections.abc.Sequence), isinstance(deck, collections.abc.Sequence)

(True, True)

AHA. The notion of registration gets `issubclss` and `isinstance` to work. But even with registration, nothing is inherited: see the mro:

In [27]:
FrenchDeck.mro()

[__main__.FrenchDeck, object]

In [41]:
deck.index(Card(rank='7', suit='spades'))

AttributeError: 'FrenchDeck' object has no attribute 'index'

Such a class is called a **virtual subclass**, and it is not checked for ABC conformance.

### subclassing from ABCs instead of registration

Remember the `Sequence` ABC above seemed to have code...

**Despite being an ABC, it had some method implementations.**

(These are the analogs of C++'s impure virtual functions.)

This suggests that it might be worth inheriting from the ABC instead

In [42]:
class FrenchDeckTake2(collections.abc.Sequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA') 
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
                       
    def __len__(self):
        return len(self._cards)
                       
#     def __getitem__(self, position): 
#         return self._cards[position]



I purposely comment `__getitem__` out. See what happens:

In [43]:
deck = FrenchDeckTake2()

TypeError: Can't instantiate abstract class FrenchDeckTake2 with abstract methods __getitem__

AHA!

By inheriting from an ABC we get runtime checks: *if my "implementation" does not fully implement my "interface", I am in trouble*. 

In [44]:
class FrenchDeckTake3(collections.abc.Sequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA') 
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
                       
    def __len__(self):
        return len(self._cards)
                       
    def __getitem__(self, position): 
        return self._cards[position]


deck = FrenchDeckTake3()

No problems now, and I seem to have picked up some methods from the sequence ABC to boot. Notice that these must be defined in terms of the existing abstract and concrete methods and we will be good...

Subtype polymorphism now works by expanding the search from the current class to other classes in the MRO!

In [45]:
FrenchDeckTake3.mro()

[__main__.FrenchDeckTake3,
 collections.abc.Sequence,
 collections.abc.Sized,
 collections.abc.Iterable,
 collections.abc.Container,
 object]

In [46]:
deck.index(Card(rank='7', suit='spades'))#obtained from sequence implementations

5

As the coder of a concrete subclass, you can override methods inherited from ABCs with more efficient implementations. For example, __contains__ works by doing a linear search, but if you were using a sorted list or binary search tree in your derived class, you could do a binary search using `bisect`.

**So implement a "virtual subclass" by registering if you dont want to use such inherited methods...the parent ABC's wont be in the `mro()`.**

**To get runtime checks and utilize methods that might have been defined in the ABC, subclass.**

#### An interesting edge case

In [48]:
class Answer:
    def __len__(self): 
        return 42
from collections import abc
isinstance(Answer(), abc.Sized), issubclass(Answer, abc.Sized)

(True, True)

In [49]:
Answer.mro()

[__main__.Answer, object]

NO EXPLICIT INHERITANCE HERE!

From https://hg.python.org/cpython/file/3.5/Lib/_collections_abc.py#l300  we can see:

```python
class Sized(metaclass=ABCMeta):

    __slots__ = ()

    @abstractmethod
    def __len__(self):
        return 0

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Sized:
            if any("__len__" in B.__dict__ for B in C.__mro__):
                return True
        return NotImplemented
```

The `isinstance`, `issubclass` dynamically check to see what the abstract methods of the `Sized` ABC are.